In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler, LabelEncoder
import pyodbc
import pickle

In [2]:
server = '172.16.68.4'
database = 'ELCM_QADB'
username = 'Temp'
password = 'Temp@123'

In [3]:
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

In [4]:
try:
    conn = pyodbc.connect(conn_str)
    print("Connected")
except pyodbc.Error as e:
    print("Error connecting to the database:", e)
    # Handle the error accordingly, such as logging, retrying, or exiting the program
    exit()

cursor = conn.cursor()

Connected


In [5]:
try:
    # Fetching data from the database
    query = 'EXEC [ED].[GetALLEmployeeMLData]'
    df = pd.read_sql_query(query, conn)
except pd.errors.DatabaseError as e:
    print("Database Error:", e)
    # Handle the error accordingly, such as reconnecting or exiting the program
    exit()
except pd.errors.EmptyDataError:
    print("The query returned an empty dataframe. Check your SQL query.")


C:\Users\Vishal\AppData\Local\Temp\ipykernel_21640\3824703933.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [6]:
try:
    # Handling duplicates
    mentors_df = df[df['IsMentor'] == 1].drop_duplicates(subset='EmpID')
except KeyError:
    print("Column 'IsMentor' or 'EmpID' not found in the dataframe.")

In [7]:
try:
    # Preprocessing skills column
    mentors_df['Skills'] = mentors_df['Skills'].str.split(', ')
except KeyError:
    print("Column 'Skills' not found in the dataframe.")

In [8]:
try:
    # Encoding Designation column
    label = LabelEncoder()
    mentors_df['Designation'] = label.fit_transform(mentors_df['Designation'])
except KeyError:
    print("Column 'Designation' not found in the dataframe.")

In [9]:
try:
    # Preprocessing skills data for MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    expanded_skill_data = mlb.fit_transform(mentors_df['Skills'])
    expanded_skill_df = pd.DataFrame(expanded_skill_data, columns=mlb.classes_, index=mentors_df.index)
except AttributeError:
    print("Attribute 'Skills' not found in the dataframe.")

In [10]:
try:
    # Combining encoded skills with the DataFrame
    mentors_df = mentors_df.drop('Skills', axis=1).join(expanded_skill_df)
except KeyError:
    print("Column 'Skills' not found in the dataframe.")

In [11]:
try:
    # Fitting Nearest Neighbors model
    knn = NearestNeighbors(n_neighbors=5)
    knn.fit(mentors_df.drop(['EmployeeNo', 'Name', 'IsMentor'], axis=1))
except ValueError as e:
    print("Value Error:", e)

In [12]:
def recommend_mentors(employee_code, knn_model, mentors_df, mlb, label, n_recommendations=5):
    try:
        # Fetching mentee data from the database
        mentee_query = f"EXEC [ED].[GetEmployeeMLData] @Employeecode = {employee_code}"
        mentee_data = pd.read_sql_query(mentee_query, conn).iloc[:1]
    except pd.errors.DatabaseError as e:
        print("Database Error:", e)
        return None

    try:
        # Preprocessing mentee data
        mentee_data['Skills'] = mentee_data['Skills'].str.split(', ')
        mentee_skill_data = mlb.transform(mentee_data['Skills'])
        mentee_skill_df = pd.DataFrame(mentee_skill_data, columns=mlb.classes_, index=mentee_data.index)
        mentee_data['Designation'] = label.transform(mentee_data['Designation'])
        mentee_data = mentee_data.drop('Skills', axis=1).join(mentee_skill_df)
    except KeyError:
        print("Column 'Skills' or 'Designation' not found in the mentee data.")
        # Handle the error accordingly
        return None
    
    try:
        # Finding nearest neighbors
        distances, indices = knn_model.kneighbors(mentee_data.drop(['EmployeeNo', 'Name', 'IsMentor'], axis=1), n_neighbors=n_recommendations)
    except ValueError as e:
        print("Value Error:", e)
        # Handle the error accordingly
        return None

    try:
        # Fetching recommended mentors
        recommended_mentors = mentors_df.iloc[indices[0]]
        return recommended_mentors
    except IndexError:
        print("Index Error: Insufficient data to recommend mentors.")
        # Handle the error accordingly
        return None

In [13]:
# Save the model and related objects
with open('model.pkl', 'wb') as model_file:
    pickle.dump(knn, model_file)

with open('mlb.pkl', 'wb') as mlb_file:
    pickle.dump(mlb, mlb_file)

with open('label.pkl', 'wb') as label_file:
    pickle.dump(label, label_file)



In [14]:
def load_model_objects():
    with open('model.pkl', 'rb') as model_file:
        knn_model = pickle.load(model_file)

    with open('mlb.pkl', 'rb') as mlb_file:
        mlb_object = pickle.load(mlb_file)

    with open('label.pkl', 'rb') as label_file:
        label_object = pickle.load(label_file)


    return knn_model, mlb_object, label_object

In [15]:
from flask import Flask, request, jsonify, render_template

In [16]:
app = Flask(__name__)
@app.route('/')
def home():
    return render_template('index1.html')




@app.route("/get_recommendations", methods=["POST"])
def get_recommendations():
    employeeCode = request.form.values()
    knn_model, mlb_object, label_object = load_model_objects()
    recommended_mentors = []
    for code in employeeCode:
        recommended_mentor = recommend_mentors(code, knn_model, mentors_df, mlb_object, label_object)
        recommended_mentors.append(recommended_mentor.head().to_dict(orient='records'))
    return jsonify(recommended_mentors)

if __name__ == "__main__":
    app.run(debug=False)



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Feb/2024 12:47:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2024 12:47:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2024 12:47:54] "GET /favicon.ico HTTP/1.1" 404 -
C:\Users\Vishal\AppData\Local\Temp\ipykernel_21640\1328717824.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mentee_data = pd.read_sql_query(mentee_query, conn).iloc[:1]
127.0.0.1 - - [15/Feb/2024 12:47:57] "POST /get_recommendations HTTP/1.1" 200 -
C:\Users\Vishal\AppData\Local\Temp\ipykernel_21640\1328717824.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mentee_data = pd.read_sql_query(mentee_query, conn).iloc[:1]
127.0.0.